In [2]:
import numpy as np
import random
from random import randint
import string
from math import pi
import math
import time
import matplotlib.pyplot as plt
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, execute, Aer, IBMQ, QuantumRegister,ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.quantum_info import Statevector, random_statevector, state_fidelity
from qiskit.aqua.utils import tensorproduct
from qiskit.providers.ibmq import least_busy
from qiskit.providers.aer.noise import NoiseModel, ReadoutError
from qiskit.providers.aer.noise.device import basic_device_readout_errors
from qiskit.circuit.library import MCMT, RYGate
import pandas as pd
# Loading your IBM Q account(s)
provider = IBMQ.load_account()


In [6]:
# dcount=dresult.get_counts(dqc)

In [10]:

# print(count2list(dcount))
# # for i in dcount.keys():
# #     print(i)
# #     print(dcount[i])

['00']


In [8]:
def exp(n,times,nshot,way,flip):
    shot_times=times*nshot
    dqc=QuantumCircuit(n,n)
    qc=QuantumCircuit(n+1,1)
    c=1
    
    dqc.measure([i for i in range(n)], [i for i in range(n)])
    t=[2*np.arccos(1/(np.sqrt(2))**i) for i in range(2**n)]
    for i in range(1,2**n):
        for j in range(n):
            if format(i, "b").zfill(n)[n-1-j]=='0':
                qc.x(j)
        qc.compose(MCMT(RYGate(t[i]),n,1),range(n+1),inplace=True)
        for j in range(n):
            if format(i, "b").zfill(n)[n-1-j]=='0':
                qc.x(j)
    qc.measure([n], [0])
    #qc.draw()
    
    device=provider.get_backend('ibmq_16_melbourne')
    device=provider.get_backend('ibmq_belem')
    if way=='noisy_simulator':
        simulator = Aer.get_backend('aer_simulator')
        noise1=NoiseModel.from_backend(device)
#         noise1= NoiseModel()
#         A=np.eye(2)*(1-2*flip)+flip*np.ones((2,2))
#         noise1.add_all_qubit_readout_error(ReadoutError(A))
        dresult=simulator.run(assemble(transpile(dqc, backend=device,optimization_level=3),\
        shots=shot_times),noise_model=noise1, memory=True).result()
        oresult=simulator.run(assemble(transpile(qc, backend=device,optimization_level=3),\
        shots=shot_times),noise_model=noise1, memory=True).result()
    elif way=='ibmq':
        #device=provider.get_backend('ibmq_belem')
        #device = least_busy(provider.backends(simulator=False, operational=True))
        dresult=device.run(assemble(transpile(dqc, backend=device,optimization_level=3),\
        shots=shot_times), memory=True).result()
        oresult=device.run(assemble(transpile(qc, backend=device,optimization_level=3),\
        shots=shot_times), memory=True).result()
    elif way=='simulator':
        simulator = Aer.get_backend('qasm_simulator')
        dresult=simulator.run(dqc, shots=shot_times, memory=True).result()
        oresult=simulator.run(qc, shots=shot_times, memory=True).result()
        
#     dlist=dresult.get_memory(dqc)
#     olist=oresult.get_memory(qc)
    dcount=dresult.get_counts(dqc)
    ocount=oresult.get_counts(qc)
    dlist=count2list(dcount)
    olist=count2list(ocount)
    derr=[]
    oerr=[]
    for i in range(times):
        dcheck_list=dlist[i:shot_times:times]
        ocheck_list=olist[i:shot_times:times]
        da1=0
        B=0
        a=''.join(["%s" % 0 for num in range(0, n)])
        if a in dcheck_list:
            B=dcheck_list.count(a)/nshot
            da1=1
        A=0
        if '0' in ocheck_list:
            A=ocheck_list.count('0')/nshot
        a1=0
        if A>=1/abs(c):
            a1=1
        ans=1
        derr.append(abs(ans-da1))
        oerr.append(abs(ans-a1))
    return derr,oerr
#np.mean(derr),np.mean(oerr),Auncertainty(derr),Auncertainty(oerr)
def count2list(c):
    l=[]
    for i in c.keys():
        for j in range(c[i]):
            l.append(i)
    return l
def exp2(n,ltimes,stimes,nshot,way,flip):
    d=[]
    o=[]
    for i in range(ltimes):
        a,b=exp(n,stimes,nshot,way,flip)
        d=d+a
        o=o+b
    #return np.mean(d),np.mean(o),Auncertainty(d),Auncertainty(o)
    print(np.mean(d))
    print(np.mean(o))
    ddata={r"$\eta$":flip,"error rate":d,r"$n":n,"method":"direct"}
    odata={r"$\eta$":flip,"error rate":o,r"$n":n,"method":"our"}
    ddf=pd.DataFrame(ddata)
    odf=pd.DataFrame(odata)
    df = ddf.append(odf)
    return df
def Auncertainty(array):
    n=len(array)
    return 1.96*np.std(array,ddof=1)/np.sqrt(n)
def makedata(n,ltimes,stimes,nshot,way,flip):
    data=pd.DataFrame()
    if type(flip)==list:
        for i in flip:
            df=exp2(n,ltimes,stimes,nshot,way,i)
            data=data.append(df)
    elif type(nshot)==list:
        for i in nshot:
            df=exp2(n,ltimes,stimes,i,way,flip)
            data=data.append(df)
    elif type(n)==list:
        for i in n:
            df=exp2(i,ltimes,stimes,nshot,way,flip)
            data=data.append(df)
    else:
        df=exp2(n,ltimes,stimes,nshot,way,flip)
        data=data.append(df)
    data.to_csv("data-{n}-{ltimes}-{stimes}-{nshot}-{way}-{flip}-{time}.csv".format(n=n,ltimes=ltimes,stimes=stimes,\
    nshot=nshot,way=way,flip=flip,time=time.time()))
    return data
def paint(n,ltimes,stimes,nshot,way,flip):
    #used when exp2 returns mean and uncertainty
    de=[]
    oe=[]
    db=[]
    ob=[]
    if type(flip)==list:
        for i in flip:
            a,b,c,d=exp2(n,ltimes,stimes,nshot,way,i)
            de.append(a)
            oe.append(b)
            db.append(c)
            ob.append(d)
        plt.errorbar(x=flip, y=de, yerr=db,color='red',label="direct")
        plt.errorbar(x=flip, y=oe, yerr=ob,color='blue',label="our")
        plt.legend()
    elif type(nshot)==list:
        for i in nshot:
            a,b,c,d=exp2(n,ltimes,stimes,i,way,flip)
            de.append(a)
            oe.append(b)
            db.append(c)
            ob.append(d)
        plt.errorbar(x=nshot, y=de, yerr=db,color='red',label="direct")
        plt.errorbar(x=nshot, y=oe, yerr=ob,color='blue',label="our")
        plt.legend()
    elif type(n)==list:
        for i in n:
            a,b,c,d=exp2(i,ltimes,stimes,nshot,way,flip)
            de.append(a)
            oe.append(b)
            db.append(c)
            ob.append(d)
        plt.errorbar(x=n, y=de, yerr=db,color='red',label="direct")
        plt.errorbar(x=n, y=oe, yerr=ob,color='blue',label="our")
        plt.legend()
    else:
        a,b,c,d=exp2(n,ltimes,stimes,nshot,way,flip)
        de.append(a)
        oe.append(b)
        db.append(c)
        ob.append(d)
    np.savez("data-{n}-{ltimes}-{stimes}-{nshot}-{way}-{flip}-{time}".format(n=n,ltimes=ltimes,stimes=stimes,\
    nshot=nshot,way=way,flip=flip,time=time.time()),de=de,oe=oe,db=db,ob=ob) 
    return de,oe,db,ob

ltimes=1
stimes=1000
nshot=1
flip_interval=[0.01*i for i in range(10)]
flip=0.05
n_interval=[i for i in range(2,5)]
n=10
data=makedata(n_interval,ltimes,stimes,nshot,'noisy_simulator',flip);

#print(data)
#paint(2,ltimes,stimes,nshot,'ibmq',flip)
#paint(2,ltimes,stimes,nshot,'qasm_simulator',flip)
#paint(2,ltimes,stimes,nshot,'noisy_simulator',flip)

0.11
0.309


In [ ]:
data=makedata(n_interval,ltimes,stimes,nshot,'ibmq',flip);

In [9]:
provider.backends(simulator=False, operational=True)

[<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [12]:
provider.backends(filters=lambda x: x.configuration().n_qubits >= 5
                                    and not x.configuration().simulator
                                    and x.status().operational==True)

[<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>]